<a href="https://colab.research.google.com/github/jonkrohn/ML-foundations/blob/master/notebooks/5-probability.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Probability & Information Theory

This class, *Probability & Information Theory*, introduces the mathematical fields that enable us to quantify uncertainty as well as to make predictions despite uncertainty. These fields are essential because machine learning algorithms are both trained by imperfect data and deployed into noisy, real-world scenarios they haven’t encountered before. 

Through the measured exposition of theory paired with interactive examples, you’ll develop a working understanding of variables, probability distributions, metrics for assessing distributions, and graphical models. You’ll also learn how to use information theory to measure how much meaningful signal there is within some given data. The content covered in this class is itself foundational for several other classes in the *Machine Learning Foundations* series, especially *Intro to Statistics* and *Optimization*.

Over the course of studying this topic, you'll: 

* Develop an understanding of what’s going on beneath the hood of predictive statistical models and machine learning algorithms, including those used for deep learning. 
* Understand the appropriate variable type and probability distribution for representing a given class of data, as well as the standard techniques for assessing the relationships between distributions.
* Apply information theory to quantify the proportion of valuable signal that’s present amongst the noise of a given probability distribution. 


**Note that this Jupyter notebook is not intended to stand alone. It is the companion code to a lecture or to videos from Jon Krohn's [Machine Learning Foundations](https://github.com/jonkrohn/ML-foundations) series, which offer detail on the following:**

*Segment 1: Introduction to Probability*
* A Brief History of Probability Theory: Frequentists vs Bayesians
* Applications of Probability to Machine Learning
* Random vs Fixed Variables
* Discrete vs Continuous Variables
* Probability Mass and Probability Density Functions
* Marginal and Conditional Probabilities
* Expected Value
* Measures of Central Tendency: Mean, Median, and Mode
* Quantiles: Quartiles, Deciles, and Percentiles
* The Box-and-Whisker Plot
* Variance and Standard Deviation
* Covariance and Correlation
* Directed and Undirected Graphical Models

*Segment 2: Distributions in Machine Learning*
* Uniform
* Gaussian: Normal and Standard Normal
* Log-Normal
* Poisson
* Exponential and Laplace
* Bernoulli, Multinomial and Multinoulli
* Mixtures of Distributions
* The Central Limit Theorem

*Segment 3: Information Theory*
* What Information Theory Is
* Nats vs Bits
* Shannon vs Differential Entropy
* Kullback-Liebler Divergence
* Cross-Entropy


**Code coming in June 2020... Watch this space**